Applying Optimization and Grid Search

In [ ]:
#imports
import itertools
import pandas as pd

# Genetic Algorithm
from library.GA import genetic_algorithm, get_best_individual 

# Class GA Solution
from library.lineup import LUGASolution

# Mutation Functions
from library.mutation_funcs import block_rotation_mutation, two_phase_shuffle_mutation, semi_shuffle

# Crossover Functions
from library.xo_funcs import cyclic_crossover, partially_mapped_crossover
# 
# Selection Algorithm
from library.selection_algorithms import tournament_selection #, linear_ranking_selection

# Line-up Problem: Datasets
from library.lineup import artists_df, conflicts_df, HERE

ImportError: cannot import name 'linear_ranking_selection' from 'library.selection_algorithms' (c:\Rafael_Bernardo\Mestrado_Data_Science_AA_DS\2_Semestre\Computational_Intelligence_for_Optimization\CIFO_PROJECT\library\selection_algorithms.py)

In [2]:
# With the final combination of logic operators and selection (with its respectives probabilities)
POP_SIZE = 10

initial_population = [LUGASolution(mutation_function = block_rotation_mutation, crossover_function = cyclic_crossover
                                   )
                      for i in range(POP_SIZE)]

for elitism in [True, False]:
    print(f"\nElistism set to {elitism}:")
    ga = genetic_algorithm(initial_population=initial_population,
                    max_gen=10, #200
                    selection_algorithm = tournament_selection,
                    maximization=True,
                    xo_prob=0.8,
                    mut_prob=1,
                    elitism=elitism,
                    verbose=False)[0]
    print(ga)


Elistism set to True:


Generations: 100%|██████████| 10/10 [00:02<00:00,  3.84gen/s]


<LUSolution (5 Stages, 7 Slots)>
╒═════════╤═════════════════════════╤═══════════════════════╤═════════════════════╤══════════════════════════╤════════════════════╤══════════════════╤══════════════════╕
│         │         Slot 1          │        Slot 2         │       Slot 3        │          Slot 4          │       Slot 5       │      Slot 6      │      Slot 7      │
╞═════════╪═════════════════════════╪═══════════════════════╪═════════════════════╪══════════════════════════╪════════════════════╪══════════════════╪══════════════════╡
│ Stage 1 │      Golden Ember       │    The Silver Owls    │   The Jazz Nomads   │ The Polyrhythm Syndicate │ Electric Serpents  │   Solar Flare    │ Nightfall Sonata │
├─────────┼─────────────────────────┼───────────────────────┼─────────────────────┼──────────────────────────┼────────────────────┼──────────────────┼──────────────────┤
│ Stage 2 │     Rhythm Alchemy      │    Mystic Rhythms     │  Cosmic Frequency   │       Echo Chamber       │   Phan

Generations: 100%|██████████| 10/10 [00:02<00:00,  3.46gen/s]


<LUSolution (5 Stages, 7 Slots)>
╒═════════╤════════════════════╤═════════════════════════╤═══════════════════╤═════════════════════╤═════════════════╤══════════════════════════╤═══════════════════════╕
│         │       Slot 1       │         Slot 2          │      Slot 3       │       Slot 4        │     Slot 5      │          Slot 6          │        Slot 7         │
╞═════════╪════════════════════╪═════════════════════════╪═══════════════════╪═════════════════════╪═════════════════╪══════════════════════════╪═══════════════════════╡
│ Stage 1 │ The Sonic Drifters │ The Bassline Architects │   Echo Chamber    │  Nightfall Sonata   │ Hypnotic Echoes │       Blue Horizon       │    Rhythm Alchemy     │
├─────────┼────────────────────┼─────────────────────────┼───────────────────┼─────────────────────┼─────────────────┼──────────────────────────┼───────────────────────┤
│ Stage 2 │ Parallel Dimension │      Turbo Vortex       │  Shadow Cadence   │ The Wandering Notes │  Neon Reverie   

In [3]:
def comb_tuning(experience_name:str, mut_func: dict, xo_func: dict, selection_algo: list,
                n_runs: int = 30, max_gen:int = 100, pop_size: int = 50):
    
    """
    Returns a DataFrame with the following columns:
    - 'Combination' - Combination tuple of a mutation, a crossover and a selection algorithm
    - 'Run' - Each Combination is runned a specified amount of times, this column specifies the run 
    number in which the results were obtained.
    - 'Generation' - Each Run is composed of a specified amount of generations, this column specifies the 
    generation number in which the results were obtained.
    - 'Best Fitness' - This is the best fitness obtained at each generation.

    Purpose:
    With this df we can play around inter-run and intra-run statistics, since it contains 
    all the results from the various combinations
    """

    combinations = list(itertools.product(list(mut_func.keys()), list(xo_func.keys()), selection_algo))

    combs_results = []
    for comb in combinations:
        # Store the best results of each generation for each run.
        runs_results = []
        for run in range(1, n_runs+1):
            initial_population = [LUGASolution(mutation_function = comb[0], crossover_function = comb[1])
                                               for i in range(pop_size)]
            
            _, generation_df = genetic_algorithm(initial_population=initial_population,
                    max_gen=max_gen,
                    selection_algorithm = comb[2],
                    maximization=True,
                    xo_prob=xo_func[comb[1]],
                    mut_prob= mut_func[comb[0]],
                    elitism=True,
                    verbose=False)
            
            current_run = pd.DataFrame()
            current_run['Run'] = [run for _ in range(1, max_gen+1)] #fill column 'Run' with the #run it is in
            
            #with run=#run, merge the df that contains the columns with #generation and its best fitness
            one_run = pd.concat([current_run, generation_df], axis=1) 
            runs_results.append(one_run)
        
        current_comb_all_runs = pd.concat(runs_results) #concat all the different runs along the index axis
        
        current_comb = pd.DataFrame()
        current_comb['Combination'] = [(comb[0].__name__, comb[1].__name__, comb[2].__name__)
                                       for _ in range(current_comb_all_runs.shape[0])]
        #combination ID (initials of mutation, xo and selection)
        current_comb['Combination ID'] = [str(comb[0].__name__[0]+comb[1].__name__[0]+comb[2].__name__[0]).upper() 
                                          for _ in range(current_comb_all_runs.shape[0])]
        one_comb = pd.concat([current_comb.reset_index(drop=True), current_comb_all_runs.reset_index(drop=True)], axis=1)
        combs_results.append(one_comb)
    
    final_results = pd.concat(combs_results).reset_index(drop=True)
    final_filepath = HERE.parent / "combination_search"/ f"final_results_{experience_name}.csv"
    final_results.to_csv(final_filepath, index=False)
    
    return final_results

In [ ]:
#Experience 1
mut_func1 = {block_rotation_mutation:0.1, two_phase_shuffle_mutation: 0.1, semi_shuffle: 0.1}
xo_func1 = {cyclic_crossover:0.8, partially_mapped_crossover: 0.8}
selection_algo1 = [tournament_selection, linear_ranking_selection]#, fitness_proportionate_selection]

#Experience 2
mut_func2 = {block_rotation_mutation:0.2, two_phase_shuffle_mutation: 0.2, semi_shuffle: 0.35}
xo_func2 = {cyclic_crossover:0.8, partially_mapped_crossover: 0.8}
selection_algo2 = [tournament_selection]

In [6]:
#experimentar 1ª com poucas runs e poucas gen e pop baixa para ver se corre tudo bem
comb_tuning(experience_name='exp1', mut_func = mut_func1, xo_func = xo_func1, selection_algo=selection_algo1,
            n_runs=2, max_gen=2, pop_size=5) #30 runs, max_gen = 100, pop_size = 100 

Gen  1:   0%|          | 0/5 [00:00<?, ?ind/s]

Generations: 100%|██████████| 2/2 [00:00<00:00, 10.52gen/s]


,Combination,Combination ID,Run,Generation,Fitness
0,"(block_rotation_mutation, cyclic_crossover, to...",BCT,1,1,1.13557
1,"(block_rotation_mutation, cyclic_crossover, to...",BCT,1,2,1.13557
2,"(block_rotation_mutation, cyclic_crossover, to...",BCT,2,1,1.10838
3,"(block_rotation_mutation, cyclic_crossover, to...",BCT,2,2,1.20733
4,"(block_rotation_mutation, partially_mapped_cro...",BPT,1,1,1.12671
5,"(block_rotation_mutation, partially_mapped_cro...",BPT,1,2,1.14171
6,"(block_rotation_mutation, partially_mapped_cro...",BPT,2,1,1.0501
7,"(block_rotation_mutation, partially_mapped_cro...",BPT,2,2,1.25796
8,"(two_phase_shuffle_mutation, cyclic_crossover,...",TCT,1,1,1.12319
9,"(two_phase_shuffle_mutation, cyclic_crossover,...",TCT,1,2,1.14519


In [76]:
results = pd.read_csv(HERE.parent /'combination_search' /'final_results.csv')
results

,Combination,Run,Generation,Fitness
0,"('block_rotation_mutation', 'cyclic_crossover'...",1,1,1.10433
1,"('block_rotation_mutation', 'cyclic_crossover'...",1,2,1.13724
2,"('block_rotation_mutation', 'cyclic_crossover'...",2,1,1.03710
3,"('block_rotation_mutation', 'cyclic_crossover'...",2,2,1.03710
4,"('block_rotation_mutation', 'custom_pmxo', 'to...",1,1,1.11533
5,"('block_rotation_mutation', 'custom_pmxo', 'to...",1,2,1.11533
6,"('block_rotation_mutation', 'custom_pmxo', 'to...",2,1,1.21196
7,"('block_rotation_mutation', 'custom_pmxo', 'to...",2,2,1.21496
8,"('two_phase_shuffle_mutation', 'cyclic_crossov...",1,1,1.09662
9,"('two_phase_shuffle_mutation', 'cyclic_crossov...",1,2,1.16371


In [65]:
results['Combination'].unique()
#len(results['Combination'].unique())

array(["('block_rotation_mutation', 'cyclic_crossover', 'tournament_selection')",
       "('block_rotation_mutation', 'cyclic_crossover', 'fitness_proportionate_selection')",
       "('block_rotation_mutation', 'custom_pmxo', 'tournament_selection')",
       "('block_rotation_mutation', 'custom_pmxo', 'fitness_proportionate_selection')",
       "('two_phase_shuffle_mutation', 'cyclic_crossover', 'tournament_selection')",
       "('two_phase_shuffle_mutation', 'cyclic_crossover', 'fitness_proportionate_selection')",
       "('two_phase_shuffle_mutation', 'custom_pmxo', 'tournament_selection')",
       "('two_phase_shuffle_mutation', 'custom_pmxo', 'fitness_proportionate_selection')",
       "('semi_shuffle', 'cyclic_crossover', 'tournament_selection')",
       "('semi_shuffle', 'cyclic_crossover', 'fitness_proportionate_selection')",
       "('semi_shuffle', 'custom_pmxo', 'tournament_selection')",
       "('semi_shuffle', 'custom_pmxo', 'fitness_proportionate_selection')"],
      dtype